In [1]:
from bs4 import BeautifulSoup
import requests,time,os
import json,re

In [2]:
# adjust the url for the each page
def get_all_song_url(pg_no):
    url = 'https://sinhalasongbook.com/all-sinhala-song-lyrics-and-chords/?_page={}/'.format(pg_no)
    return url

In [3]:
# wirte the song links to song_links.csv file
def write_res(links_array):
    with open('test1.csv','a',newline='') as f:
        for link in links_array:
            f.write(link + os.linesep)

In [4]:
# get the response of a http request
def make_req(url):
    headers = requests.utils.default_headers()
    response = requests.get(url, headers)
    return response.text

In [5]:
# parse the song links pages using BeautifulSoup
def parse_html(html_pg):
    links = []
    soup = BeautifulSoup(html_pg, 'html.parser')
    song_links = soup.find_all("a",{"class": "_blank"})
    for tag in song_links:
        link = tag.get('href')
        links.append(link)
    return links

In [6]:
# get the urls of all the songs
def get_song_list():
    for pg_no in range(1, 3):
        url = get_all_song_url(pg_no)
        html_pg = make_req(url)
        links_array = parse_html(html_pg)
        write_res(links_array)

In [7]:
get_song_list()

In [7]:
# read the song linknss file
def get_song_url(line_num):
    with open('test1.csv', 'r') as f:
        lines = f.readlines()
    return lines[line_num]

In [8]:
# get_song_url(12)

In [9]:
# BeautifulSoup(make_req('https://sinhalasongbook.com/aaley/'), 'html.parser')

In [10]:
from googletrans import Translator
def parse_html_song(html_pg):
    translator = Translator()
    with open ('next_song_link.txt', 'r') as f:
        next_index = int(f.readlines()[0])
    soup = BeautifulSoup(html_pg, 'html.parser')
    song = {}
    class_list = ["entry-tags","entry-categories","entry-author-name","lyrics","music"]
    title = soup.find('h1', {"class": "entry-title"}).get_text()
    views = soup.find('div',{'class':'tptn_counter'}).get_text().split()[1].split('Visits')[0]
    song.update({'id' : next_index })
    song.update({'title': title})
    song.update({'views': int(views.replace(',',''))})
    for class_l in class_list:
        content = soup.find_all('span',{"class":class_l})
        if content:
            key, val = process_content(content[0])
#             val = translator.translate(val, src = 'en',dest='si')

            if ((not key is None) and (not val is None)):
                if(type(val)== list):
                    newVal =[]
                    for value in val:
                        newVal.append(translator.translate(value, src='en',dest='si').text)
                        song.update({key:newVal})
                        print(key , val)
                else:
#                     print(key , val)
                    song.update({key:translator.translate(val, src='en',dest='si').text})

        else:
            pass
    unprocessed_lyrics = soup.select('pre')[0].get_text()
    processed_lyrics = parse_lyrics(unprocessed_lyrics)
    song.update({'song_lyrics': processed_lyrics})
    print(song)
    return song

In [11]:
def process_content(key_val_pair):
    if key_val_pair:
        key_val_pair = key_val_pair.get_text()
        split_pair=key_val_pair.split(':')
        if len(split_pair)>1:
            key = split_pair[0]
            val = split_pair[1]
            if ',' in val:
                values = []
                split_val = val.split(',')
                for value in split_val:
                    values.append(value)
                return key,values
            else:
                return key,val
        else:
            return None,None
    else:
        return None,None

def parse_lyrics(lyrics):
    space_set = set([' '])
    processed = ''
    regex = r"([A-z])+|[0-9]|\||-|∆|([.!?\\\/\(\)\+#&])+"
    lyric_lines = lyrics.split('\n')
    for line in lyric_lines:
        new = re.sub(regex, '', line)
        chars = set(new)
        if not ((chars == space_set) or (len(chars) is 0)):
            processed += new + '\n'
    return processed

In [12]:
def write_song(song,id):
    with open ('new-corpus/song_' + str(id)+'.json','w+') as f:
        f.write(json.dumps(song))

In [13]:
def scrape_songs():
    with open ('next_song_link.txt', 'r') as f:
        next_index = int(f.readlines()[0])
    while next_index < 600 :
        print('Scraping song',next_index)
        url = get_song_url(int(next_index))
        html_doc = make_req(url)
        song = parse_html_song(html_doc)
        write_song(song,next_index)
        next_index = next_index + 1
        with open('next_song_link.txt', 'w') as f:
            f.write(str(next_index))
        time.sleep (5)

In [ ]:
scrape_songs()

Scraping song 232
Genre [' Movie Songs', ' Request']
Genre [' Movie Songs', ' Request']
{'id': 232, 'title': 'Asha Dasin Dakina – ආශා දෑසින් දකිනා', 'views': 774, 'Genre': ['චිත්රපට ගීත', 'ඉල්ලීම'], 'Artist': 'දේවානන්දා Waidyasekara', 'Lyrics': 'ස්වර්ණ ශ්රී බණ්ඩාර', 'Music': 'පණ්ඩිත් අමරදේව', 'song_lyrics': '——\nආශා දෑසින් දකිනා සිහිනේ\xa0\nමිහිර මැකී යනවා\nඒ සිහිනේ පුතු තනිකම ඉපදී\xa0\nහද තුල තෙරපෙනවා\nආ මග දෑලේ පිපි මල් පරවී\xa0\nනෑ ඵල හටගන්නේ\nසිතුවිලි මී මැසි පොදි නැත මී බැඳ\xa0\nඋන්මය රිදවන්නේ\nතන්හා දෑතින් පැතු මේ වරපට\nගිලිහී බිම වැටිලා\nදෛවයෙ ගොනු ඇද ජීවන ගැල ගෙන\nආ මග දිග යනවා\nඈත අතීතයෙ සිහින මැදින් යලි\xa0\nජීවනයට යනවා\n'}
Scraping song 233
{'id': 233, 'title': 'Asha Nirasha Mawu – ආශා නිරාශා මැවූ', 'views': 2815, 'Genre': 'ගෝල්ඩන් ඩිස්කෝ', 'Artist': 'වික්ටර් රත්නායක', 'Lyrics': 'අජන්තා රණසිංහ', 'Music': 'වික්ටර් රත්නායක', 'song_lyrics': '   \xa0    \n       \xa0\nආශා නිරාශා මැවූ ඈ ස්වප්න මායාවකී\nස්නේහේ ගඟුල් ප්\u200dරේමේ තැවුල්\nමැවූ ඉන්ද්\u200dරජාලේ ඇයයි \n   \xa0    \n  

In [ ]:
# make_req('https://sinhalasongbook.com/abhisarikawange-%e0%b6%85%e0%b6%b7%e0%b7%92%e0%b7%83%e0%b7%8f%e0%b6%bb%e0%b7%92%e0%b6%9a%e0%b7%8f%e0%b7%80%e0%b6%b1%e0%b7%8a%e0%b6%9c%e0%b7%9a/')

In [ ]:
translator = Translator()
from googletrans import Translator

In [ ]:
translator = Translator()
translator.translate('Gamage Amaradewa.', src='en', dest='si').text